# Q5 差旅与费用报销异常分析

依据PDF示例提示：
- 5.1 重复报销识别：按员工+供应商+交易日+费用类型(+金额)分组，count>1视为潜在重复；
- 5.2 反复性报销：同一员工+供应商在7日窗口内次数≥3，输出窗口与计数；
- 5.3 流程异常：应收据但未收到、提交日期晚于付款日期、审批日期早于提交日期、非报销项金额>0等。

为避免超时，读取部分行进行开发，提供可视化与文字分析。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)

te = pd.read_excel('/workspace/Sample Data.Case 5.xlsx', sheet_name='T&E', nrows=80000)
for c in ['TxnDate','SubmitDate','SentForPaymentDate','ExtractDateTime','ApproverDate']:
    if c in te.columns:
        te[c] = pd.to_datetime(te[c], errors='coerce')
for c in ['EmployeeID','EmployeeName','ReportID','Vendor','ExpenseType','ReceiptRequired','ReceiptReceived','Reimbursable_YorN']:
    if c in te.columns:
        te[c] = te[c].astype(str).str.strip()

print(te.shape)
te.head(3)

## 5.1 重复报销识别

In [ ]:
keys = [k for k in ['EmployeeID','Vendor','TxnDate','ExpenseType'] if k in te.columns]
amt_col = 'Txn_Amt' if 'Txn_Amt' in te.columns else None
keys_amt = keys + ([amt_col] if amt_col else [])
dup = te.groupby(keys_amt).size().reset_index(name='count')
dup2 = dup[dup['count']>1].sort_values('count', ascending=False)
print('潜在重复组数（子集）：', dup2.shape[0])
display(dup2.head(10))

plt.figure(figsize=(8,4))
sns.histplot(dup2['count'], bins=30, color='tomato')
plt.title('重复报销组的计数分布（子集）')
plt.xlabel('重复次数')
plt.ylabel('组数')
plt.tight_layout()
plt.show()

## 5.2 7日窗口内反复性报销

In [ ]:
results = []
if {'EmployeeID','Vendor','TxnDate'}.issubset(te.columns):
    te2 = te.dropna(subset=['EmployeeID','Vendor','TxnDate']).sort_values(['EmployeeID','Vendor','TxnDate'])
    for (eid, v), df in te2.groupby(['EmployeeID','Vendor']):
        dates = df['TxnDate'].dt.normalize()
        i = 0
        n = len(dates)
        while i < n:
            j = i
            while j < n and (dates.iloc[j] - dates.iloc[i]).days <= 7:
                j += 1
            count = j - i
            if count >= 3:
                results.append({'EmployeeID':eid,'Vendor':v,'start':dates.iloc[i],'end':dates.iloc[j-1],'count':count})
            i += 1
rep = pd.DataFrame(results).sort_values('count', ascending=False)
print('7日窗口≥3次的窗口条数（子集）：', rep.shape[0])
display(rep.head(10))

plt.figure(figsize=(8,4))
sns.histplot(rep['count'], bins=30, color='seagreen')
plt.title('7日窗口内报销次数分布（子集）')
plt.xlabel('窗口内次数')
plt.ylabel('窗口数')
plt.tight_layout()
plt.show()

## 5.3 流程异常检测

In [ ]:
anom_list = []
if {'ReceiptRequired','ReceiptReceived'}.issubset(te.columns):
    a = te[(te['ReceiptRequired'].str.upper()=='Y') & (te['ReceiptReceived'].str.upper()=='N')]
    anom_list.append(('Receipt required but not received', a))
if {'SubmitDate','SentForPaymentDate'}.issubset(te.columns):
    b = te[(te['SentForPaymentDate'].notna()) & (te['SubmitDate'].notna()) & (te['SubmitDate']>te['SentForPaymentDate'])]
    anom_list.append(('Submit after payment date', b))
if {'ApproverDate','SubmitDate'}.issubset(te.columns):
    c = te[(te['ApproverDate'].notna()) & (te['SubmitDate'].notna()) & (te['ApproverDate']<te['SubmitDate'])]
    anom_list.append(('ApproverDate earlier than SubmitDate', c))
if {'Reimbursable_YorN','Txn_Amt'}.issubset(te.columns):
    d = te[(te['Reimbursable_YorN'].str.contains('N', case=False, na=False)) & (pd.to_numeric(te['Txn_Amt'], errors='coerce')>0)]
    anom_list.append(('Non-reimbursable marked but positive amount', d))

for name, df in anom_list:
    print(name, 'count:', len(df))
    cols_show = [c for c in ['EmployeeID','EmployeeName','Vendor','ExpenseType','TxnDate','SubmitDate','SentForPaymentDate','ApproverDate','ReceiptRequired','ReceiptReceived','Txn_Amt','ReportID'] if c in df.columns]
    display(df[cols_show].head(10))

# 可视化：收据缺失的分布
if anom_list:
    a = dict(anom_list).get('Receipt required but not received')
    if a is not None and len(a)>0:
        plt.figure(figsize=(8,4))
        a['month'] = a['TxnDate'].dt.to_period('M')
        cnt = a.groupby('month').size().reset_index(name='count')
        sns.lineplot(x=cnt['month'].astype(str), y='count', data=cnt, marker='o')
        plt.xticks(rotation=45, ha='right')
        plt.title('应收据未收到的月度分布（子集）')
        plt.ylabel('次数')
        plt.xlabel('月份')
        plt.tight_layout()
        plt.show()

### 结果与审计建议
- 重复与反复性报销需结合票据影像与实际行程/消费证据抽查，识别同票多报、拆分报销。
- 流程异常（收据缺失、提交晚于付款等）应回溯审批链与制度执行力，关注是否存在绕流程或内部控制薄弱。
- 可设定更严格的重复定义（如加入地点/项目/成本中心），及更合理的窗口定义（滚动窗口与非重叠窗口），降低误报。

## 附加：全量运行与更多维度分析

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
outputs_path = Path('/workspace/KPMG_HW1/outputs')
outputs_path.mkdir(parents=True, exist_ok=True)

te_all = pd.read_excel('/workspace/Sample Data.Case 5.xlsx', sheet_name='T&E')
for c in ['TxnDate','SubmitDate','SentForPaymentDate','ExtractDateTime','ApproverDate']:
    if c in te_all.columns:
        te_all[c] = pd.to_datetime(te_all[c], errors='coerce')
for c in ['EmployeeID','EmployeeName','ReportID','Vendor','ExpenseType','ReceiptRequired','ReceiptReceived','Reimbursable_YorN']:
    if c in te_all.columns:
        te_all[c] = te_all[c].astype(str).str.strip()

# 重复报销
keys = [k for k in ['EmployeeID','Vendor','TxnDate','ExpenseType'] if k in te_all.columns]
amt_col = 'Txn_Amt' if 'Txn_Amt' in te_all.columns else None
keys_amt = keys + ([amt_col] if amt_col else [])
dup = te_all.groupby(keys_amt).size().reset_index(name='count')
dup2 = dup[dup['count']>1].sort_values('count', ascending=False)
dup2.to_csv(outputs_path/'Q5_duplicates.csv', index=False)

# 7日窗口反复性
results = []
if {'EmployeeID','Vendor','TxnDate'}.issubset(te_all.columns):
    te2 = te_all.dropna(subset=['EmployeeID','Vendor','TxnDate']).sort_values(['EmployeeID','Vendor','TxnDate'])
    for (eid, v), df in te2.groupby(['EmployeeID','Vendor']):
        dates = df['TxnDate'].dt.normalize()
        i = 0
        n = len(dates)
        while i < n:
            j = i
            while j < n and (dates.iloc[j] - dates.iloc[i]).days <= 7:
                j += 1
            count = j - i
            if count >= 3:
                results.append({'EmployeeID':eid,'Vendor':v,'start':dates.iloc[i],'end':dates.iloc[j-1],'count':count})
            i += 1
rep = pd.DataFrame(results).sort_values('count', ascending=False)
rep.to_csv(outputs_path/'Q5_repetitive_windows.csv', index=False)

# 流程异常导出
anom_exports = {}
if {'ReceiptRequired','ReceiptReceived'}.issubset(te_all.columns):
    a = te_all[(te_all['ReceiptRequired'].str.upper()=='Y') & (te_all['ReceiptReceived'].str.upper()=='N')]
    anom_exports['Q5_receipt_missing.csv'] = a
if {'SubmitDate','SentForPaymentDate'}.issubset(te_all.columns):
    b = te_all[(te_all['SentForPaymentDate'].notna()) & (te_all['SubmitDate'].notna()) & (te_all['SubmitDate']>te_all['SentForPaymentDate'])]
    anom_exports['Q5_submit_after_payment.csv'] = b
if {'ApproverDate','SubmitDate'}.issubset(te_all.columns):
    c = te_all[(te_all['ApproverDate'].notna()) & (te_all['SubmitDate'].notna()) & (te_all['ApproverDate']<te_all['SubmitDate'])]
    anom_exports['Q5_approver_before_submit.csv'] = c
if {'Reimbursable_YorN','Txn_Amt'}.issubset(te_all.columns):
    d = te_all[(te_all['Reimbursable_YorN'].str.contains('N', case=False, na=False)) & (pd.to_numeric(te_all['Txn_Amt'], errors='coerce')>0)]
    anom_exports['Q5_nonreimbursable_positive.csv'] = d

for fname, df in anom_exports.items():
    df.to_csv(outputs_path/fname, index=False)

# 额外维度：按费用类型的重复与反复性统计
if 'ExpenseType' in te_all.columns:
    dup_type_counts = dup2.groupby('ExpenseType')['count'].sum().reset_index().sort_values('count', ascending=False)
    dup_type_counts.to_csv(outputs_path/'Q5_duplicate_by_expensetype.csv', index=False)

print('Q5全量导出完成：duplicates, windows, process anomalies, by expense type')